In [2]:
import nbformat
import os
import shutil

def repair_widgets_metadata(notebook_path):
    try:
        # Load notebook safely
        nb = nbformat.read(notebook_path, as_version=4)
        md = nb.metadata

        # Create backup
        backup_path = notebook_path + ".bak"
        if not os.path.exists(backup_path):
            shutil.copy2(notebook_path, backup_path)
            print(f"🧾 Backup created: {backup_path}")

        if "widgets" in md:
            widgets = md["widgets"]

            # The canonical structure we want
            expected = {
                "application/vnd.jupyter.widget-state+json": {
                    "state": {},
                    "version_major": 2,
                    "version_minor": 0
                }
            }

            # Fix cases
            if not isinstance(widgets, dict) or "application/vnd.jupyter.widget-state+json" not in widgets:
                print(f"🧹 Removing invalid widget metadata from: {notebook_path}")
                md.pop("widgets", None)
            else:
                widget_json = widgets["application/vnd.jupyter.widget-state+json"]
                # Ensure required fields
                if "state" not in widget_json:
                    print(f"🔧 Adding missing 'state' key to: {notebook_path}")
                    widget_json["state"] = {}
                if "version_major" not in widget_json:
                    widget_json["version_major"] = 2
                if "version_minor" not in widget_json:
                    widget_json["version_minor"] = 0

        # Save fixed notebook
        nbformat.write(nb, notebook_path)
    except Exception as e:
        print(f"⚠️ Error processing {notebook_path}: {e}")

def repair_all_notebooks(folder):
    for root, _, files in os.walk(folder):
        for f in files:
            if f.endswith(".ipynb"):
                repair_widgets_metadata(os.path.join(root, f))

if __name__ == "__main__":
    folder = input("Enter folder containing notebooks: ").strip()
    if not os.path.isdir(folder):
        print("❌ Invalid folder path.")
    else:
        repair_all_notebooks(folder)
        print("\n✅ Done! All notebooks repaired successfully.")


Enter folder containing notebooks: /content/drive/MyDrive/Colab Notebooks/Deepfake video generation/
🧾 Backup created: /content/drive/MyDrive/Colab Notebooks/Deepfake video generation/GAN_TOOLS.ipynb.bak
🔧 Adding missing 'state' key to: /content/drive/MyDrive/Colab Notebooks/Deepfake video generation/GAN_TOOLS.ipynb
🧾 Backup created: /content/drive/MyDrive/Colab Notebooks/Deepfake video generation/Deepfake_generation.ipynb.bak

✅ Done! All notebooks repaired successfully.


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# **First** **Order** **Model**

In [ ]:
!git clone https://github.com/AliaksandrSiarohin/first-order-model

Cloning into 'first-order-model'...
remote: Enumerating objects: 397, done.
remote: Counting objects: 100% (52/52), done.
remote: Compressing objects: 100% (18/18), done.
remote: Total 397 (delta 38), reused 34 (delta 34), pack-reused 345 (from 3)
Receiving objects: 100% (397/397), 72.19 MiB | 49.18 MiB/s, done.
Resolving deltas: 100% (205/205), done.


In [ ]:
cd first-order-model

/content/first-order-model


In [ ]:
import imageio
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from skimage.transform import resize
from IPython.display import HTML
import warnings
warnings.filterwarnings("ignore")

source_image = imageio.imread('/content/drive/MyDrive/Colab Notebooks/Deepfake video generation/real_00002.jpg')
reader = imageio.get_reader('/content/drive/MyDrive/Colab Notebooks/OWN DATASET/REAL VIDEOS/id00060/00307.mp4')


#Resize image and video to 256x256

source_image = resize(source_image, (256, 256))[..., :3]

fps = reader.get_meta_data()['fps']
driving_video = []
try:
    for im in reader:
        driving_video.append(im)
except RuntimeError:
    pass
reader.close()

driving_video = [resize(frame, (256, 256))[..., :3] for frame in driving_video]

def display(source, driving, generated=None):
    fig = plt.figure(figsize=(8 + 4 * (generated is not None), 6))

    ims = []
    for i in range(len(driving)):
        cols = [source]
        cols.append(driving[i])
        if generated is not None:
            cols.append(generated[i])
        im = plt.imshow(np.concatenate(cols, axis=1), animated=True)
        plt.axis('off')
        ims.append([im])

    ani = animation.ArtistAnimation(fig, ims, interval=50, repeat_delay=1000)
    plt.close()
    return ani


HTML(display(source_image, driving_video).to_html5_video())

In [ ]:
!pip install ffmpeg-python

In [ ]:
from demo import load_checkpoints
generator, kp_detector = load_checkpoints(config_path='/content/drive/MyDrive/Colab Notebooks/Deepfake video generation/First Order Model/vox-256.yaml',
                            checkpoint_path='/content/drive/MyDrive/Colab Notebooks/Deepfake video generation/First Order Model/vox-adv-cpk.pth.tar')

In [ ]:
from demo import make_animation
from skimage import img_as_ubyte
import imageio
from IPython.display import HTML
# from demo import display  # Ensure this is your display function - Remove this line
import os

# ========== Parameters ==========
fps = 30  # Set the frame rate of your output video
output_path = '/content/generated.mp4'  # You can change this as needed

# ========== Generate Animation ==========
predictions = make_animation(
    source_image,
    driving_video,
    generator,
    kp_detector,
    relative=True
)

# ========== Save the Video ==========
imageio.mimsave(output_path, [img_as_ubyte(frame) for frame in predictions], fps=fps)

# ========== Display in Notebook (Optional) ==========
HTML(display(source_image, driving_video, predictions).to_html5_video())



  0%|          | 0/122 [00:00<?, ?it/s]

In [ ]:
from google.colab import files
files.download('/content/generated.mp4')

In [ ]:



from demo import make_animation
from skimage import img_as_ubyte
import imageio
from IPython.display import HTML
# from demo import display  # Ensure this is your display function - Remove this line
import os

# ========== Parameters ==========
fps = 30  # Set the frame rate of your output video
output_path = '/content/generated_2.mp4'  # You can change this as needed

# ========== Generate Animation ==========
predictions = make_animation(source_image, driving_video, generator, kp_detector, relative=False, adapt_movement_scale=True)

# ========== Save the Video ==========
imageio.mimsave(output_path, [img_as_ubyte(frame) for frame in predictions], fps=fps)

# ========== Display in Notebook (Optional) ==========
HTML(display(source_image, driving_video, predictions).to_html5_video())




  0%|          | 0/122 [00:00<?, ?it/s]

In [ ]:
!pip install ffmpeg-python

In [ ]:
import os
import imageio
import numpy as np
from skimage.transform import resize
from tqdm import tqdm
import shutil
import warnings

warnings.filterwarnings("ignore")

# Set paths
images_folder = '/content/drive/MyDrive/Colab Notebooks/OWN DATASET/REAL IMAGE'
videos_folder = '/content/drive/MyDrive/Colab Notebooks/OWN DATASET/flat_real_videos'
output_folder = '/content/drive/MyDrive/Colab Notebooks/OWN DATASET/generated_deepfakes_video'

# Create output folder
os.makedirs(output_folder, exist_ok=True)

# Load model
from demo import load_checkpoints, make_animation

generator, kp_detector = load_checkpoints(
    config_path='/content/drive/MyDrive/Colab Notebooks/Deepfake video generation/First Order Model/vox-256.yaml',
    checkpoint_path='/content/drive/MyDrive/Colab Notebooks/Deepfake video generation/First Order Model/vox-adv-cpk.pth.tar'
)

# Define supported video extensions
VIDEO_EXTENSIONS = ['.mp4', '.avi', '.mov', '.mkv']

# Get list of images and videos
image_files = sorted([
    os.path.join(images_folder, f) for f in os.listdir(images_folder)
    if f.lower().endswith(('.jpg', '.jpeg', '.png'))
])

video_files = sorted([
    f for f in os.listdir(videos_folder)
    if os.path.isfile(os.path.join(videos_folder, f)) and os.path.splitext(f)[1].lower() in VIDEO_EXTENSIONS
])

# Check if image list is empty
if not image_files:
    raise ValueError("No images found in the image folder!")

# Loop over all videos and assign an image in round-robin fashion
for idx, video_name in enumerate(tqdm(video_files, desc="Generating deepfakes")):
    video_path = os.path.join(videos_folder, video_name)

    # Select image in round-robin
    image_path = image_files[idx % len(image_files)]

    try:
        # Load and resize source image
        source_image = imageio.imread(image_path)
        source_image = resize(source_image, (256, 256))[..., :3]

        # Read video
        reader = imageio.get_reader(video_path)
        fps = reader.get_meta_data()['fps']
        driving_video = []
        try:
            for frame in reader:
                driving_video.append(resize(frame, (256, 256))[..., :3])
        except RuntimeError:
            print(f"Skipping video {video_name} due to read error.")
            continue
        finally:
            reader.close()

        if not driving_video:
            print(f"Skipping {video_name}, no frames read.")
            continue

        # Generate animation
        predictions = make_animation(source_image, driving_video, generator, kp_detector, relative=True)

        # Save output
        output_path = os.path.join(output_folder, f"deepfake_{os.path.splitext(video_name)[0]}.mp4")
        imageio.mimsave(output_path, [np.uint8(frame * 255) for frame in predictions], fps=fps)

    except Exception as e:
        print(f"Error processing {video_name}: {e}")
        continue


Generating deepfakes:   0%|          | 0/95 [00:00<?, ?it/s]

  0%|          | 0/378 [00:00<?, ?it/s]

Generating deepfakes:   1%|          | 1/95 [00:12<19:10, 12.24s/it]

  0%|          | 0/107 [00:00<?, ?it/s]

Generating deepfakes:   2%|▏         | 2/95 [00:15<11:09,  7.20s/it]

...

Generating deepfakes:  98%|█████████▊| 93/95 [08:31<00:12,  6.29s/it]

  0%|          | 0/296 [00:00<?, ?it/s]

Generating deepfakes:  99%|█████████▉| 94/95 [08:41<00:07,  7.29s/it]

  0%|          | 0/133 [00:00<?, ?it/s]

Generating deepfakes: 100%|██████████| 95/95 [08:46<00:00,  5.54s/it]
